# Filter Itemid Chart

This script is used for filtering itemids from TABLE CHARTEVENTS.

1. We check number of units of each itemid and choose the major unit as the target of unit conversion.
2. In this step we get 3 kinds of features:
    - numerical features
    - categorical features
    - ratio features, this usually happens in blood pressure measurement, such as "135/70".

## Output

1. itemid of observations for chartevents.
2. unit of measurement for each itemid.

In [1]:
from __future__ import print_function

import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
from multiprocessing import Pool, cpu_count
import re

from utils import getConnection

%matplotlib inline

In [2]:
conn = getConnection()
    
_adm = np.load('res/admission_ids.npy').tolist()
admission_ids = _adm['admission_ids']
admission_ids_txt = _adm['admission_ids_txt']

db = np.load('res/itemids.npy').tolist()
input_itemid = db['input']
output_itemid = db['output']
chart_itemid = db['chart']
lab_itemid = db['lab']
microbio_itemid = db['microbio']
prescript_itemid = db['prescript']

In [3]:
def stat_chart_unit_task(i, admission_ids_txt):
    tconn = getConnection()
    
    # for each itemID select number of rows group by unit of measurement.
    tcur = tconn.cursor()
    tcur.execute('SELECT coalesce(valueuom, \'\'), count(*) FROM mimiciii.chartevents WHERE itemid = '+ str(i) +' and hadm_id in (select * from admission_ids) group by valueuom')
    chartunits = tcur.fetchall()
    chartunits = sorted(chartunits, key=lambda tup: tup[1])
    chartunits.reverse()
    
    # count number of observation that has non numeric value
    tcur = tconn.cursor()
    tcur.execute('SELECT count(*) FROM mimiciii.chartevents WHERE itemid = '+ str(i) +' and hadm_id in (select * from admission_ids) and valuenum is null')
    notnum = tcur.fetchone()
    notnum = notnum[0]
    
    # total number of observation
    tcur = tconn.cursor()
    tcur.execute('SELECT count(*) FROM mimiciii.chartevents WHERE itemid = '+ str(i) +' and hadm_id in (select * from admission_ids)')
    total = tcur.fetchone()
    total = total[0]
    
    tconn.close()
    return (i, chartunits, notnum, total)

p = Pool(cpu_count() * 2)
results = [p.apply_async(stat_chart_unit_task, args=(i, admission_ids_txt)) for i in chart_itemid]
p.close()
p.join()
results = [x.get() for x in results]
np.save('res/filtered_chart_raw.npy', {'raw': results})
print('saved!')

saved!


## First filtering of categorical features

All features with numerical values < 80% of all records are possible categorical features. In this step we drop them for later analyzing.

In [4]:
results = np.load('res/filtered_chart_raw.npy').tolist()['raw']
valid_chart = []
valid_chart_unit = []
valid_chart_cate = []
valid_chart_num = []
dropped = []
multiple_units = []
for x in results:
    i, chartunits, notnum, total = x[0], x[1], x[2], x[3]
    
    # calculate percentage of the top frequent unit compared to all observation.
    total2 = 0
    unitnum = 0
    for c in chartunits:
        total2 += c[1]
        if c[0] != '':
            unitnum += 1
    if total2 == 0:
        continue
    percentage = float(chartunits[0][1]) / total2 * 100.
    if unitnum > 1:
        multiple_units.append((i, chartunits, percentage))
    print("CHART "+str(i) + "\t" + "{:.2f}".format(percentage) +'\t'+ str(chartunits))
    
    # if the percentage of numeric number is less, then dropped it, and make it categorical feature.
    percentage =float(total -notnum)*100 / total
    print("Numeric observation :" + "{:.4f}%".format(percentage)+ " ( NOTNUM= " + str(notnum) + " / ALL= " + str(total) + " ) ")
    if(percentage < 80): 
        print('dropped\n')
        dropped.append(i)
        continue;
    print('')
    valid_chart.append(i)
    valid_chart_unit.append(chartunits[0][0])

CHART 2848	100.00	[('', 32)]
Numeric observation :0.0000% ( NOTNUM= 32 / ALL= 32 ) 
dropped

CHART 3565	100.00	[('', 59508)]
Numeric observation :0.0000% ( NOTNUM= 59508 / ALL= 59508 ) 
dropped

CHART 2026	100.00	[('', 158)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 158 ) 

CHART 6158	100.00	[('', 1)]
Numeric observation :0.0000% ( NOTNUM= 1 / ALL= 1 ) 
dropped

CHART 3028	100.00	[('', 1)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 1 ) 

CHART 2409	100.00	[('', 48)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 48 ) 

CHART 225771	100.00	[('', 519)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 519 ) 

CHART 4349	100.00	[('', 36)]
Numeric observation :0.0000% ( NOTNUM= 36 / ALL= 36 ) 
dropped

CHART 4718	100.00	[('', 53)]
Numeric observation :0.0000% ( NOTNUM= 53 / ALL= 53 ) 
dropped

CHART 3370	100.00	[('', 38505)]
Numeric observation :0.0000% ( NOTNUM= 38505 / ALL= 38505 ) 
dropped

CHART 4466	100.00	[('', 11)]
Numeric observation :0.0000% ( NOTNUM= 11 / ALL= 1

Numeric observation :96.6956% ( NOTNUM= 476 / ALL= 14405 ) 

CHART 4740	100.00	[('', 30)]
Numeric observation :0.0000% ( NOTNUM= 30 / ALL= 30 ) 
dropped

CHART 4887	100.00	[('', 17)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 17 ) 

CHART 6023	100.00	[('', 7)]
Numeric observation :0.0000% ( NOTNUM= 7 / ALL= 7 ) 
dropped

CHART 6280	100.00	[('', 55)]
Numeric observation :87.2727% ( NOTNUM= 7 / ALL= 55 ) 

CHART 4643	100.00	[('', 30)]
Numeric observation :0.0000% ( NOTNUM= 30 / ALL= 30 ) 
dropped

CHART 6947	100.00	[('', 3)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 3 ) 

CHART 228005	100.00	[('ml/hr', 41505)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 41505 ) 

CHART 220073	100.00	[('mmHg', 37288)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 37288 ) 

CHART 223932	100.00	[('', 23522)]
Numeric observation :0.0000% ( NOTNUM= 23522 / ALL= 23522 ) 
dropped

CHART 2685	100.00	[('', 41)]
Numeric observation :97.5610% ( NOTNUM= 1 / ALL= 41 ) 

CHART 3292	100.00	[(''


CHART 3559	100.00	[('', 420055)]
Numeric observation :0.0000% ( NOTNUM= 420055 / ALL= 420055 ) 
dropped

CHART 4266	100.00	[('', 28)]
Numeric observation :0.0000% ( NOTNUM= 28 / ALL= 28 ) 
dropped

CHART 6127	100.00	[('', 5)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 5 ) 

CHART 1165	100.00	[('', 1)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 1 ) 

CHART 226780	100.00	[('', 17)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 17 ) 

CHART 228396	100.00	[('', 39605)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 39605 ) 

CHART 224923	100.00	[('', 73139)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 73139 ) 

CHART 7132	100.00	[('', 121)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 121 ) 

CHART 3423	100.00	[('', 1727)]
Numeric observation :0.0000% ( NOTNUM= 1727 / ALL= 1727 ) 
dropped

CHART 7638	100.00	[('', 2)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 2 ) 

CHART 6069	100.00	[('', 1)]
Numeric observation :0.0000% ( NOTNUM= 1 / ALL= 1 ) 
dropp

CHART 409	100.00	[('', 22)]
Numeric observation :0.0000% ( NOTNUM= 22 / ALL= 22 ) 
dropped

CHART 7116	100.00	[('', 80)]
Numeric observation :98.7500% ( NOTNUM= 1 / ALL= 80 ) 

CHART 986	100.00	[('', 281)]
Numeric observation :99.6441% ( NOTNUM= 1 / ALL= 281 ) 

CHART 3011	100.00	[('', 13)]
Numeric observation :0.0000% ( NOTNUM= 13 / ALL= 13 ) 
dropped

CHART 2107	100.00	[('', 41)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 41 ) 

CHART 691	100.00	[('', 72518)]
Numeric observation :0.0000% ( NOTNUM= 72518 / ALL= 72518 ) 
dropped

CHART 3627	100.00	[('', 443520)]
Numeric observation :0.0000% ( NOTNUM= 443520 / ALL= 443520 ) 
dropped

CHART 224114	100.00	[('', 6730)]
Numeric observation :0.0000% ( NOTNUM= 6730 / ALL= 6730 ) 
dropped

CHART 1949	100.00	[('', 61)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 61 ) 

CHART 6060	100.00	[('', 11)]
Numeric observation :0.0000% ( NOTNUM= 11 / ALL= 11 ) 
dropped

CHART 226739	100.00	[('years', 18)]
Numeric observation :100.0000% ( NOT

CHART 8436	100.00	[('', 37524)]
Numeric observation :0.0000% ( NOTNUM= 37524 / ALL= 37524 ) 
dropped

CHART 223760	100.00	[('', 11732)]
Numeric observation :0.0000% ( NOTNUM= 11732 / ALL= 11732 ) 
dropped

CHART 4483	100.00	[('', 42)]
Numeric observation :0.0000% ( NOTNUM= 42 / ALL= 42 ) 
dropped

CHART 6578	100.00	[('', 9)]
Numeric observation :0.0000% ( NOTNUM= 9 / ALL= 9 ) 
dropped

CHART 227459	100.00	[('ug/mL', 8)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 8 ) 

CHART 2990	100.00	[('', 17)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 17 ) 

CHART 7655	100.00	[('', 36)]
Numeric observation :0.0000% ( NOTNUM= 36 / ALL= 36 ) 
dropped

CHART 110	100.00	[('', 2255)]
Numeric observation :0.0000% ( NOTNUM= 2255 / ALL= 2255 ) 
dropped

CHART 3522	100.00	[('', 108)]
Numeric observation :0.0000% ( NOTNUM= 108 / ALL= 108 ) 
dropped

CHART 5958	100.00	[('', 1)]
Numeric observation :0.0000% ( NOTNUM= 1 / ALL= 1 ) 
dropped

CHART 1342	100.00	[('', 19)]
Numeric observation :10.5263


CHART 4735	100.00	[('', 51)]
Numeric observation :0.0000% ( NOTNUM= 51 / ALL= 51 ) 
dropped

CHART 224997	100.00	[('', 836)]
Numeric observation :0.0000% ( NOTNUM= 836 / ALL= 836 ) 
dropped

CHART 1614	100.00	[('', 15)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 15 ) 

CHART 227471	100.00	[('', 17199)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 17199 ) 

CHART 225135	100.00	[('', 29348)]
Numeric observation :0.0000% ( NOTNUM= 29348 / ALL= 29348 ) 
dropped

CHART 1369	100.00	[('', 1)]
Numeric observation :0.0000% ( NOTNUM= 1 / ALL= 1 ) 
dropped

CHART 3817	100.00	[('', 1821)]
Numeric observation :1.3729% ( NOTNUM= 1796 / ALL= 1821 ) 
dropped

CHART 8509	100.00	[('', 58)]
Numeric observation :0.0000% ( NOTNUM= 58 / ALL= 58 ) 
dropped

CHART 223931	100.00	[('', 30199)]
Numeric observation :0.0000% ( NOTNUM= 30199 / ALL= 30199 ) 
dropped

CHART 1627	100.00	[('', 1)]
Numeric observation :0.0000% ( NOTNUM= 1 / ALL= 1 ) 
dropped

CHART 5213	100.00	[('', 106)]
Numeric observatio


CHART 5068	100.00	[('', 29)]
Numeric observation :0.0000% ( NOTNUM= 29 / ALL= 29 ) 
dropped

CHART 224025	100.00	[('', 2312)]
Numeric observation :0.0000% ( NOTNUM= 2312 / ALL= 2312 ) 
dropped

CHART 3160	100.00	[('', 98)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 98 ) 

CHART 4601	100.00	[('', 13)]
Numeric observation :0.0000% ( NOTNUM= 13 / ALL= 13 ) 
dropped

CHART 309	100.00	[('', 3545)]
Numeric observation :33.5120% ( NOTNUM= 2357 / ALL= 3545 ) 
dropped

CHART 795	100.00	[('', 13704)]
Numeric observation :99.1097% ( NOTNUM= 122 / ALL= 13704 ) 

CHART 1651	100.00	[('', 37)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 37 ) 

CHART 349	100.00	[('', 1753)]
Numeric observation :0.0000% ( NOTNUM= 1753 / ALL= 1753 ) 
dropped

CHART 226713	100.00	[('', 6963)]
Numeric observation :0.0000% ( NOTNUM= 6963 / ALL= 6963 ) 
dropped

CHART 4560	100.00	[('', 547)]
Numeric observation :0.0000% ( NOTNUM= 547 / ALL= 547 ) 
dropped

CHART 1096	100.00	[('', 2)]
Numeric observation :100.0

dropped

CHART 2952	100.00	[('', 57)]
Numeric observation :3.5088% ( NOTNUM= 55 / ALL= 57 ) 
dropped

CHART 221	100.00	[('', 147143)]
Numeric observation :12.6673% ( NOTNUM= 128504 / ALL= 147143 ) 
dropped

CHART 223982	100.00	[('', 147218)]
Numeric observation :0.0000% ( NOTNUM= 147218 / ALL= 147218 ) 
dropped

CHART 227730	100.00	[('', 2261)]
Numeric observation :0.0000% ( NOTNUM= 2261 / ALL= 2261 ) 
dropped

CHART 2124	100.00	[('', 4)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 4 ) 

CHART 6618	100.00	[('', 1)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 1 ) 

CHART 224733	100.00	[('', 361834)]
Numeric observation :0.0000% ( NOTNUM= 361834 / ALL= 361834 ) 
dropped

CHART 227943	100.00	[('', 630)]
Numeric observation :0.0000% ( NOTNUM= 630 / ALL= 630 ) 
dropped

CHART 6125	100.00	[('', 13)]
Numeric observation :76.9231% ( NOTNUM= 3 / ALL= 13 ) 
dropped

CHART 226758	100.00	[('', 18)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 18 ) 

CHART 6243	100.00	[('', 7)]
Num

Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 8765 ) 

CHART 1824	100.00	[('', 12)]
Numeric observation :0.0000% ( NOTNUM= 12 / ALL= 12 ) 
dropped

CHART 1781	100.00	[('', 19)]
Numeric observation :0.0000% ( NOTNUM= 19 / ALL= 19 ) 
dropped

CHART 6289	100.00	[('', 6)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 6 ) 

CHART 224928	100.00	[('', 7491)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 7491 ) 

CHART 224899	100.00	[('cm', 1847)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 1847 ) 

CHART 725	92.04	[('cmH2O', 474), ('', 41)]
Numeric observation :91.6505% ( NOTNUM= 43 / ALL= 515 ) 

CHART 129	100.00	[('', 30725)]
Numeric observation :0.0000% ( NOTNUM= 30725 / ALL= 30725 ) 
dropped

CHART 5195	100.00	[('', 48)]
Numeric observation :14.5833% ( NOTNUM= 41 / ALL= 48 ) 
dropped

CHART 227700	100.00	[('', 292992)]
Numeric observation :0.0000% ( NOTNUM= 292992 / ALL= 292992 ) 
dropped

CHART 1106	100.00	[('', 12)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 12 

## Unit inconsistency

Here are itemids having two or more different units.

For [211, 505], they have the same unit in fact. Keep them.

For [3451, 578, 113], the major unit covers > 90% of all records. Keep them.

For [3723], it is just a typo and we keep all.

In [5]:
for i, chartunits, percentage in sorted(multiple_units, key=lambda x: x[2]):
    total2 = sum([t[1] for t in chartunits])
    percentage = float(chartunits[0][1]) / total2 * 100.
    print("CHART "+str(i) + "\t" + "{:.4f}".format(percentage) +'\t'+ str(chartunits))

CHART 3723	59.3440	[('cm', 258375), ('kg', 177010)]
CHART 211	67.2600	[('BPM', 3484614), ('bpm', 1696195)]
CHART 578	94.0624	[('.', 277532), ('cmH20', 12941), ('', 4578)]
CHART 3451	94.3170	[('kg', 179206), ('cm', 10793), ('', 5)]
CHART 113	97.4122	[('mmHg', 1167662), ('%', 23460), ('', 7559)]
CHART 227441	99.3007	[('mg/dL', 142), ('units', 1)]
CHART 505	99.4428	[('cmH20', 350365), ('', 1958), ('cmH2O', 5)]


In [6]:
dropped_id = dropped
print(dropped_id, len(dropped_id))

[2848, 3565, 6158, 4349, 4718, 3370, 4466, 223837, 224890, 496, 8394, 6122, 4888, 2311, 6967, 599, 3323, 3071, 2769, 4321, 6169, 225099, 1820, 4875, 224854, 7477, 2627, 5729, 359, 225003, 3322, 228307, 227837, 1478, 3367, 1543, 7003, 1229, 227677, 6312, 2150, 3483, 7981, 4650, 224889, 3666, 4722, 224692, 8519, 1116, 224960, 4388, 8087, 5142, 7697, 7818, 5796, 2452, 227570, 6960, 6068, 5605, 5080, 537, 5099, 2300, 7214, 224450, 8372, 8178, 227782, 4475, 6680, 325, 223991, 223947, 1999, 227834, 1333, 6782, 4986, 3124, 5561, 5048, 225958, 3510, 7300, 7088, 5329, 4910, 227683, 3335, 6309, 227808, 228153, 8399, 227752, 1389, 1364, 227783, 655, 224084, 167, 6163, 224991, 5656, 478, 8523, 224988, 458, 4414, 629, 223782, 3564, 1606, 2761, 4355, 2693, 227240, 227929, 4360, 6210, 3492, 5347, 224633, 1000, 227474, 227961, 2542, 1963, 5210, 6118, 228448, 77, 225743, 140, 4976, 56, 6790, 3384, 4291, 8466, 224101, 228449, 4449, 5762, 7934, 224976, 227963, 224005, 223840, 712, 228446, 1910, 224935, 8

In [7]:
def numerical_ratio(units):
    res = list(map(lambda unit: re.match(r'(\d+\.\d*)|(\d*\.\d+)|(\d+)', unit), units))
    numerical_ratio = 1.0 * len([1 for r in res if r is not None]) / len(res)
    return numerical_ratio

In [8]:
dropped_value = []
for d in dropped_id:
    print('LAB : ' + str(d))
    cur = conn.cursor()
    cur.execute('SELECT value, valueuom, count(*) as x FROM mimiciii.chartevents as lb \
                WHERE itemid = '+ str(d) +' and hadm_id in (select * from admission_ids) GROUP BY value, valueuom ORDER BY x DESC')
    droped_outs = cur.fetchall()
    drop_array = []
    ct =0
    total = 0;
    for dx in droped_outs:
        total += dx[2]
    print("Count ",total)
    print([d[0] for d in droped_outs])
    print('Numeric ratio', numerical_ratio([str(d[0]) for d in droped_outs]))
    units = []
    for dx in droped_outs:
        ct+=1
        if(ct>20):
            break
        dx = list(dx)
        print(dx[1],dx[0],"\t",dx[2])
    print('')
    dropped_value.append((d, droped_outs))
    
np.save('res/chart_dropped_value.npy', dropped_value)

LAB : 2848
Count  32
['intact', 'DSD', None, 'WNL']
Numeric ratio 0.0
None intact 	 27
None DSD 	 3
None None 	 1
None WNL 	 1

LAB : 3565
Count  59508
['Yes', None, 'Comment', 'No']
Numeric ratio 0.0
None Yes 	 59175
None None 	 241
None Comment 	 82
None No 	 10

LAB : 6158
Count  1
['drawn']
Numeric ratio 0.0
None drawn 	 1

LAB : 4349
Count  36
['Given', 'given', None]
Numeric ratio 0.0
None Given 	 21
None given 	 12
None None 	 3

LAB : 4718
Count  53
[None, 'cd&i', 'cd+i', 'D+I', 'done', 'intact', 'd+i', 'Done', 'CD&I', 'dry', 'D&I', 'd&i', 'c/di/i']
Numeric ratio 0.0
None None 	 27
None cd&i 	 11
None cd+i 	 3
None D+I 	 2
None done 	 2
None intact 	 1
None d+i 	 1
None Done 	 1
None CD&I 	 1
None dry 	 1
None D&I 	 1
None d&i 	 1
None c/di/i 	 1

LAB : 3370
Count  38505
['crib', None, 'isolette', 'warmer', 'yes']
Numeric ratio 0.0
None crib 	 38068
None None 	 261
None isolette 	 158
None warmer 	 16
None yes 	 2

LAB : 4466
Count  11
['given']
Numeric ratio 0.0
None given 	 1

Count  17179
['Open to Air', 'Dry Sterile Dsg', 'Duoderm', 'Other/Remarks', 'Wet to Dry', None, 'Aquacel', 'Transparent', 'Ace Wrap', 'Allevyn', 'Adaptic', 'Wound Gel', 'Aquaphor', 'Aquacel AG', 'Antifungal Oint', 'Hydrogel/Vigilon', 'Collagenase', 'Alginate/Kalstat', 'Wound Gel/Adapti', 'Antifungal Powde', 'Dermagran', 'Wnd Gel/Allevyn', 'Accuzyme']
Numeric ratio 0.0
None Open to Air 	 8478
None Dry Sterile Dsg 	 3347
None Duoderm 	 1197
None Other/Remarks 	 997
None Wet to Dry 	 550
None None 	 542
None Aquacel 	 487
None Transparent 	 399
None Ace Wrap 	 260
None Allevyn 	 213
None Adaptic 	 209
None Wound Gel 	 160
None Aquaphor 	 115
None Aquacel AG 	 78
None Antifungal Oint 	 48
None Hydrogel/Vigilon 	 30
None Collagenase 	 23
None Alginate/Kalstat 	 20
None Wound Gel/Adapti 	 8
None Antifungal Powde 	 6

LAB : 225003
Count  741
['Not assessed', 'Open', 'Staples', 'Approximated', 'Sutures', 'Steri-strip']
Numeric ratio 0.0
None Not assessed 	 233
None Open 	 229
None Staples 	 87

Count  36044
['Dry and intact', 'Changed', 'Removed', 'Not assessed', 'Reinforced']
Numeric ratio 0.0
None Dry and intact 	 20746
None Changed 	 8798
None Removed 	 4908
None Not assessed 	 1137
None Reinforced 	 455

LAB : 4475
Count  17
['given', 'Given']
Numeric ratio 0.0
None given 	 14
None Given 	 3

LAB : 6680
Count  22
['given', 'Given', 'instill']
Numeric ratio 0.0
None given 	 16
None Given 	 4
None instill 	 2

LAB : 325
Count  226826
['WNL', None, 'Infiltrated', 'Redness', 'Bleeding', 'Other/Remarks', 'Ecchymotic', 'Drainage', 'Pain', 'Swelling', 'Erythematous']
Numeric ratio 0.0
None WNL 	 207889
None None 	 10951
None Infiltrated 	 2438
None Redness 	 1524
None Bleeding 	 1220
None Other/Remarks 	 874
None Ecchymotic 	 715
None Drainage 	 541
None Pain 	 417
None Swelling 	 214
None Erythematous 	 43

LAB : 223991
Count  337744
['Strong', 'Weak', 'None']
Numeric ratio 0.0
None Strong 	 180033
None Weak 	 104539
None None 	 53172

LAB : 223947
Count  262214
['Easily Palpab

Count  150828
['Systolic', 'Mean', 'Diastolic']
Numeric ratio 0.0
None Systolic 	 141880
None Mean 	 8715
None Diastolic 	 233

LAB : 227929
Count  105
['Sit', 'Supine', 'Stand']
Numeric ratio 0.0
None Sit 	 59
None Supine 	 44
None Stand 	 2

LAB : 4360
Count  22
['given', 'Given', 'applied']
Numeric ratio 0.0
None given 	 15
None Given 	 5
None applied 	 2

LAB : 6210
Count  1
['sent']
Numeric ratio 0.0
None sent 	 1

LAB : 3492
Count  216
['Given', None]
Numeric ratio 0.0
None Given 	 202
None None 	 14

LAB : 5347
Count  13
['given', 'Applied', 'applied', 'done', 'Given']
Numeric ratio 0.0
None given 	 4
None Applied 	 4
None applied 	 3
None done 	 1
None Given 	 1

LAB : 224633
Count  63723
['No', 'Not assessed', 'Yes']
Numeric ratio 0.0
None No 	 49315
None Not assessed 	 12522
None Yes 	 1886

LAB : 1000
Count  877
['30', '14', '30mg', '20', '25mg', '25', '30MG', '25mg/hr', '2', '4', '16', '8', '10', '14mg/hr', '16mg/hr', '20mg', '30mg/hr', '25MG', '1.5', '6mg', '6mg/hr', '3.5'

Count  1028
['given', 'Given', None, 'GIVEN', 'dip pac', 'yes', 'given.', 'sucrose', 'givn', 'gvien']
Numeric ratio 0.0
None given 	 637
None Given 	 210
None None 	 139
None GIVEN 	 31
None dip pac 	 4
None yes 	 3
None given. 	 1
None sucrose 	 1
None givn 	 1
None gvien 	 1

LAB : 8433
Count  20438
['Intact', 'Changed', 'Reinforced']
Numeric ratio 0.0
None Intact 	 16873
None Changed 	 3407
None Reinforced 	 158

LAB : 376
Count  76703
['Coccyx', 'Arm, Left Lower', 'Arm, Right Lower', 'Leg, Left Lower', 'Scrotom', 'Abdomen', 'Perineum', 'Leg, Right Lower', 'Heel, Left', 'Facial', 'Back, Upper', 'Leg, Left Upper', 'Head', 'Gluteal, Left', 'Heel, Right', 'Arm, Left Upper', 'Leg, Right Upper', 'Arm, Right Upper', 'Chest', 'Back', 'Foot, Left', 'Gluteal, Right', 'Extremities, Lo', 'Hand, Left', 'Hand, Right', 'Hip, Left', 'Left knee', 'Torso', 'Toes', 'Neck', 'Back, Lower', 'Foot, Right', 'Penis', 'Hip, Right', None, 'Shoulder, Left', 'Ankle, Left', 'Ankle, Right', 'Right knee', 'Elbow,

...

Count  114743
['Post line placement', 'On admission', 'Not applicable']
Numeric ratio 0.0
None Post line placement 	 100817
None On admission 	 8212
None Not applicable 	 5714

LAB : 2786
Count  12
['DONE', 'done', None]
Numeric ratio 0.0
None DONE 	 7
None done 	 4
None None 	 1

LAB : 223998
Count  32378
['Negative', 'Positive', 'Not Applicable']
Numeric ratio 0.0
None Negative 	 29669
None Positive 	 2412
None Not Applicable 	 297

LAB : 8193
Count  14
['done', 'Given', 'given', 'GIVEN']
Numeric ratio 0.0
None done 	 8
None Given 	 4
None given 	 1
None GIVEN 	 1

LAB : 5907
Count  198627
['Yes', None, 'No', 'Comment']
Numeric ratio 0.0
None Yes 	 198408
None None 	 154
None No 	 59
None Comment 	 6

LAB : 223996
Count  38342
['Serosanguinous', 'Serous', 'Sanguinous', 'None', 'Straw', 'Purulent', 'Bilious', 'Clear']
Numeric ratio 0.0
None Serosanguinous 	 25519
None Serous 	 5748
None Sanguinous 	 4243
None None 	 1622
None Straw 	 1050
None Purulent 	 106
None Bilious 	 50
None Cle

Count  120766
['Normal', 'Shallow', 'Labored', 'Dyspneic', 'Other/Remarks', None, 'Apneic', 'Agonal', 'Splinting']
Numeric ratio 0.0
None Normal 	 106686
None Shallow 	 6070
None Labored 	 3991
None Dyspneic 	 1774
None Other/Remarks 	 1281
None None 	 605
None Apneic 	 142
None Agonal 	 131
None Splinting 	 86

LAB : 5200
Count  18
['applied', 'done', 'Applied', 'appiled', 'given', 'Done']
Numeric ratio 0.0
None applied 	 11
None done 	 2
None Applied 	 2
None appiled 	 1
None given 	 1
None Done 	 1

LAB : 453
Count  420
['Absent', 'Present', None]
Numeric ratio 0.0
None Absent 	 344
None Present 	 51
None None 	 25

LAB : 1548
Count  16
['on', '0n']
Numeric ratio 0.5
None on 	 15
None 0n 	 1

LAB : 3668
Count  616
['1 Mild/Disturbed', '2 Mod/Severe', None]
Numeric ratio 0.6666666666666666
None 1 Mild/Disturbed 	 541
None 2 Mod/Severe 	 74
None None 	 1

LAB : 228164
Count  7
['Right', 'Left']
Numeric ratio 0.0
None Right 	 4
None Left 	 3

LAB : 3145
Count  19
['+']
Numeric ratio 0.

Count  45568
['Negative', 'Positive', None, 'Not Applicable', 'Other/Remarks']
Numeric ratio 0.0
None Negative 	 38896
None Positive 	 5738
None None 	 792
None Not Applicable 	 126
None Other/Remarks 	 16

LAB : 5639
Count  12
['INTACT', 'intact', 'site ok', None, 'SITE OK']
Numeric ratio 0.0
None INTACT 	 5
None intact 	 3
None site ok 	 2
None None 	 1
None SITE OK 	 1

LAB : 223993
Count  40203
['Mediastinal', 'Left Anterior', 'Right Anterior', 'Left posterior', 'Right Posterior']
Numeric ratio 0.0
None Mediastinal 	 14903
None Left Anterior 	 10076
None Right Anterior 	 9307
None Left posterior 	 3037
None Right Posterior 	 2880

LAB : 228419
Count  1111
['No', 'Yes']
Numeric ratio 0.0
None No 	 885
None Yes 	 226

LAB : 8072
Count  40
['CDI', 'cdi', None, 'c,d,i', 'C&D', 'c/d/i', 'c&d', 'CD']
Numeric ratio 0.0
None CDI 	 22
None cdi 	 10
None None 	 2
None c,d,i 	 2
None C&D 	 1
None c/d/i 	 1
None c&d 	 1
None CD 	 1

LAB : 4994
Count  10
['done']
Numeric ratio 0.0
None done 	 1

Count  331376
['None', 'Small', 'Medium', 'Large', None]
Numeric ratio 0.0
None None 	 289469
None Small 	 26423
None Medium 	 9773
None Large 	 5049
None None 	 662

LAB : 3690
Count  53070
['Yes', 'No', None, 'Comment']
Numeric ratio 0.0
None Yes 	 34786
None No 	 16115
None None 	 2155
None Comment 	 14

LAB : 6300
Count  1
['NEG']
Numeric ratio 0.0
None NEG 	 1

LAB : 223971
Count  6498
['L Femoral', 'R Femoral', 'R Radial', 'L Radial', 'R Brachial ', 'L Brachial']
Numeric ratio 0.0
None L Femoral 	 4705
None R Femoral 	 1296
None R Radial 	 247
None L Radial 	 125
None R Brachial  	 75
None L Brachial 	 50

LAB : 2868
Count  15
['125/74', '121/71', '120/68', '125/75', '118/66', '121/73', '136/86', '120/67', '126/74', '124/72', '120/76', '115/63', '126/75', '115/69']
Numeric ratio 1.0
None 125/74 	 2
None 121/71 	 1
None 120/68 	 1
None 125/75 	 1
None 118/66 	 1
None 121/73 	 1
None 136/86 	 1
None 120/67 	 1
None 126/74 	 1
None 124/72 	 1
None 120/76 	 1
None 115/63 	 1
None 126

Count  3824
['Negative', 'Positive', 'Not Applicable']
Numeric ratio 0.0
None Negative 	 1564
None Positive 	 1419
None Not Applicable 	 841

LAB : 224911
Count  31302
['None', 'Small', 'Scant', 'Moderate', 'Not assessed', 'Large', 'Copious']
Numeric ratio 0.0
None None 	 17010
None Small 	 5259
None Scant 	 4640
None Moderate 	 2596
None Not assessed 	 737
None Large 	 636
None Copious 	 424

LAB : 8344
Count  18
['given', 'Given', '1.7 meq']
Numeric ratio 0.3333333333333333
None given 	 14
None Given 	 3
None 1.7 meq 	 1

LAB : 227831
Count  535
['Dependent', 'max A', 'Independent', 'min A', 'mod A', 'Supervision', 'Contact guarding']
Numeric ratio 0.0
None Dependent 	 150
None max A 	 114
None Independent 	 83
None min A 	 80
None mod A 	 48
None Supervision 	 43
None Contact guarding 	 17

LAB : 224990
Count  5998
['No', 'Not assessed', 'Yes']
Numeric ratio 0.0
None No 	 5233
None Not assessed 	 682
None Yes 	 83

LAB : 227620
Count  2763
['Intact', 'Erythema', 'Excoriated', 'Not a

In [9]:
print(len(dropped_id))
print(len(valid_chart), len(valid_chart_unit))
print(valid_chart)
print(valid_chart_unit)

3819
2644 2644
[2026, 3028, 2409, 225771, 6873, 2961, 2265, 1101, 227597, 8533, 6810, 225118, 224144, 227366, 3652, 228344, 227056, 5587, 6466, 1060, 26, 8549, 227346, 220739, 227021, 5565, 220277, 224178, 1726, 2640, 1487, 786, 220580, 226730, 227916, 7245, 2755, 3142, 541, 1582, 4198, 226746, 6003, 227429, 142, 226753, 6585, 227537, 226892, 3045, 7027, 2019, 6465, 6987, 8446, 227000, 226137, 6170, 1982, 220587, 2572, 8305, 71, 624, 626, 3242, 823, 1528, 8484, 8491, 1374, 1981, 6286, 2925, 2363, 228099, 225070, 228374, 2522, 211, 7632, 7493, 6964, 225112, 224953, 228146, 226772, 1966, 8048, 2373, 5865, 6129, 224054, 8301, 224674, 2112, 227543, 2339, 3082, 8362, 6657, 7534, 227872, 2992, 2783, 1542, 8517, 1551, 226980, 2717, 224367, 7353, 227448, 6058, 223765, 429, 224754, 226113, 3029, 2480, 2687, 6906, 7650, 2692, 3161, 6768, 226993, 6643, 530, 228336, 1475, 224684, 224675, 8061, 1978, 223763, 228163, 3663, 6409, 226775, 2353, 227039, 224332, 5558, 223848, 7262, 803, 7982, 494, 2856,

## Store selected features in first filtering

These features are all numerical features.

In [10]:
np.save('res/filtered_chart.npy',{'id':valid_chart,'unit':valid_chart_unit})
# np.save('res/filtered_chart_cate',{'id':[223758],'unit':None})
print('saved!')

saved!


## Divide dropped features in first filtering

- Features with the ratio of non-numerical values(values that cannot pass the parser) > 0.5: categorical features
- Features with the ratio of ratio values > 0.5: ratio features
- otherwise: (possible) numerical features, we will parse them later

In [11]:
dropped_value = np.load('res/chart_dropped_value.npy').tolist()
valid_chart_num = []
valid_chart_num_unit = []
valid_chart_cate = []
valid_chart_ratio = []
for d, droped_outs in dropped_value:
    ascnum = 0
    rationum = 0
    for value, valueuom, count in droped_outs:
        value = str(value)
        isasc = re.search(r'(\d+\.\d*)|(\d*\.\d+)|(\d+)', value) is None
        isratio = re.fullmatch(r'{0}\/{0}'.format(r'((\d+\.\d*)|(\d*\.\d+)|(\d+))'), value) is not None
        if isasc:
            ascnum += 1
        if isratio:
            rationum += 1
    if ascnum / len(droped_outs) >= 0.5:
        valid_chart_cate.append(d)
    elif rationum / len(droped_outs) >= 0.5:
        valid_chart_ratio.append(d)
        print(droped_outs)
    else:
        valid_chart_num.append(d)
        if droped_outs[0][1] is None:
            valid_chart_num_unit.append('')
        else:
            valid_chart_num_unit.append(droped_outs[0][1])
#         print(droped_outs)
        
print(len(valid_chart_num), len(valid_chart_cate), len(valid_chart_ratio))
print(valid_chart_num, valid_chart_num_unit, valid_chart_ratio)

[('127/58', None, 3), ('132/61', None, 1), ('137/63', None, 1), ('128/53', None, 1), ('135/60', None, 1), ('140/68', None, 1), ('124/58', None, 1), ('130/59', None, 1), ('129/62', None, 1), ('157/67', None, 1), ('134/68', None, 1), ('215/91', None, 1), ('116/56', None, 1), ('110/57', None, 1), ('124/59', None, 1), ('144/60', None, 1), ('185/81', None, 1), ('122/58', None, 1), ('156/62', None, 1), ('129/58', None, 1), ('143/69', None, 1), ('138/61', None, 1), ('140/66', None, 1), ('122/54', None, 1), ('115/55', None, 1), ('133/56', None, 1), ('139/62', None, 1), ('120/56', None, 1), ('120/58', None, 1), ('143/62', None, 1), ('117/53', None, 1), ('124/54', None, 1), ('156/63', None, 1), ('120/57', None, 1), ('127/57', None, 1), ('130/60', None, 1), ('121/60', None, 1), ('123/57', None, 1), ('138/68', None, 1)]
[('110/46', None, 1), ('112/45', None, 1), ('119/52', None, 1), ('127/56', None, 1), ('126/56', None, 1), ('105/45', None, 1), ('111/42', None, 1), ('119/54', None, 1), ('129/59', 

## Store 3 kinds of features

In [12]:
print(len(valid_chart_num), len(valid_chart_num_unit), len(valid_chart_cate))
print(valid_chart_num, valid_chart_num_unit, valid_chart_cate)
np.save('res/filtered_chart_num',{'id':valid_chart_num,'unit':valid_chart_num_unit})
np.save('res/filtered_chart_cate',{'id':valid_chart_cate,'unit':None})
np.save('res/filtered_chart_ratio', {'id': valid_chart_ratio, 'unit': None})

314 314 3405
[223837, 496, 2311, 3323, 3322, 3483, 8087, 227570, 225958, 227808, 3564, 1000, 227961, 712, 8410, 653, 8377, 65, 804, 1233, 224181, 5931, 2445, 5943, 1057, 5668, 417, 639, 1891, 225977, 228156, 222, 562, 2131, 1886, 8383, 7081, 3751, 228016, 405, 228007, 228009, 3359, 224413, 6402, 563, 479, 8384, 1771, 2145, 1451, 1394, 459, 3109, 851, 228013, 7045, 224716, 7372, 5, 227984, 5952, 669, 708, 534, 223907, 228372, 223925, 3111, 368, 2226, 372, 2945, 224069, 1837, 228008, 606, 373, 523, 225409, 228018, 228017, 1734, 404, 556, 3594, 2730, 6114, 2349, 1412, 690, 112, 7018, 1561, 225413, 6537, 227378, 7988, 8465, 1052, 224436, 483, 1342, 28, 585, 762, 1730, 97, 3321, 369, 226152, 1413, 826, 4186, 3591, 607, 1560, 858, 2774, 2062, 857, 8400, 2519, 793, 224831, 4183, 3618, 3601, 482, 6283, 3206, 1471, 2382, 1334, 1036, 6640, 227805, 308, 1456, 3299, 8403, 223, 3320, 2056, 852, 1627, 3622, 2185, 8413, 555, 225953, 8464, 39, 228442, 1469, 5321, 225979, 102, 2, 227131, 4603, 6131, 73